In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
transactions = pd.read_csv("../../data/transactions_train.csv")
transactions.drop(['t_dat', 'price', 'sales_channel_id'], axis=1, inplace=True)

In [3]:
transactions.head()

,customer_id,article_id
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004


In [4]:
transactions.iloc[:1000].groupby(['customer_id', 'article_id']).value_counts()


In [6]:
transactions.head()

,index,customer_id,article_id
0,0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001
1,1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023
2,2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004
3,3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003
4,4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004
